# Models baseline

# 1. Imports

## 1.1 Packages

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import HistGradientBoostingRegressor

from store_sales.pipelines.model_training.nodes import create_lag_features, create_rolling_mean_features

## 1.2 Options

In [2]:
lags = [20, 30, 45, 60, 90, 120, 365, 730]
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]

## 1.3 Dataset

In [ ]:
df_training = pd.read_parquet("../data/04_feature/df_train_feature_engineered.parquet")

# df_test = pd.read_parquet("../data/04_feature/df_test_feature_engineered.parquet")

# 2. Hist Gradient Boosting model

In [ ]:
df_training.sample(2)

,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,holiday_type,is_workday,is_train,is_validation
864814,855904,2014-04-27,24,HOME AND KITCHEN II,0.0,0,NaN,Guayaquil,Guayas,D,...,False,False,False,False,False,1,None,1,1,0
972967,961387,2014-06-25,33,SCHOOL AND OFFICE SUPPLIES,0.0,0,875.0,Quevedo,Los Rios,C,...,False,False,False,False,False,2,Event_National,1,1,0


In [ ]:
df_train = df_training[df_training["is_train"] == 1]
df_valid = df_training[df_training["is_validation"] == 1]

In [ ]:
# Create lag features
df_train = create_lag_features(df_train, lags)
df_train = create_rolling_mean_features(df_train, lags)

df_valid = create_lag_features(df_valid, lags)

In [8]:
df_train.tail(5)

,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,...,sales_lag_365,sales_lag_730,rolling_mean_20,rolling_mean_30,rolling_mean_45,rolling_mean_60,rolling_mean_90,rolling_mean_120,rolling_mean_365,rolling_mean_730
3047087,2993627,2017-08-11,54,SEAFOOD,0.0,0,768.0,El Carmen,Manabi,C,...,5.0,5.0,3.45,2.966667,2.911111,3.016667,2.977778,2.875000,2.005479,1.791781
3048869,2995409,2017-08-12,54,SEAFOOD,1.0,1,903.0,El Carmen,Manabi,C,...,1.0,0.0,3.30,2.933333,2.888889,3.016667,2.933333,2.858333,1.991781,1.784932
3050651,2997191,2017-08-13,54,SEAFOOD,2.0,0,1054.0,El Carmen,Manabi,C,...,2.0,1.0,3.10,2.966667,2.888889,3.000000,2.922222,2.866667,1.991781,1.786301
3052433,2998973,2017-08-14,54,SEAFOOD,0.0,0,818.0,El Carmen,Manabi,C,...,0.0,0.0,3.15,3.033333,2.933333,2.983333,2.944444,2.841667,1.991781,1.787671
3054215,3000755,2017-08-15,54,SEAFOOD,3.0,0,802.0,El Carmen,Manabi,C,...,2.0,3.0,2.80,2.966667,2.888889,2.933333,2.944444,2.800000,1.991781,1.787671


In [ ]:
y_train = df_train["sales"]
x_train = df_train.drop(columns=["id", "date"])